<a href="https://colab.research.google.com/github/tauseeqhussain/SemanticSegmentation_CNN/blob/main/SemanticSegmentation_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from segmentation_utility.semantic_segmentation \
    import (create_semantic_segmentation_dataset, display_segmented_image, display_grayscale_array, plot_class_masks)
#from segmentation_utility.semantic_segmentation import create_semantic_segmentation_dataset, display_segmented_image, display_grayscale_array, plot_class_masks

from tensorflow import keras
from keras import layers, models
from tensorflow.keras.models import Sequential

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.layers import Conv2D, AveragePooling2D
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.layers import Activation
from tensorflow.keras.models import Model





In [ ]:
# ============================= step 1: prepare datasets =============================
tf.config.set_visible_devices([], 'GPU')  # CPU mode, comment this line to switch GPU mode

(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()  # load MNIST data

# set how many digits you are going to recognize and segment. If 3, then digits 0,1,2 will be used.
n_classes = 3

# preprocess MNIST data for semantic segmentation
# train_x: training data, train_y: training labels, test_x: validation data, test_y: validation labels
train_x, train_y, test_x, test_y = create_semantic_segmentation_dataset(num_train_samples=5000,
                                                                        num_test_samples=1000,
                                                                        image_shape=(60, 60),
                                                                        num_classes=n_classes)

# display one example, note the semantic data is randomly generated.
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
display_grayscale_array(train_x[0], ax=ax1, title='Input image')
display_segmented_image(train_y[0], ax=ax2, title='Ground truth')
plt.show()

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# ============================= step 2: construct the model here =============================
#model = tf.keras.Sequential([])  # construct your model according to the given Table

model = Sequential([
    Conv2D(16, 5, padding='same', activation='relu', input_shape=(60, 60,1)),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(32, 3, padding='same', activation='relu'),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(64, 3, padding='same', activation='relu'),
    Conv2D(64, 3, padding='same', activation='relu'),
    Conv2DTranspose(64,3,strides=2, padding='same', activation='relu'),
    Conv2DTranspose(3,3,strides=2, padding='same', activation='sigmoid')
])
#checking my model summary
model.summary()

In [ ]:
# ============================= step 3: compile and train =============================

model.compile(optimizer=Adam(learning_rate=0.01),loss=BinaryCrossentropy(from_logits=True),metrics=['accuracy'])
#model.compile()  # set up your training hyperparameters, use Adam for optimizer and 0.01 as learning rate
# use BinaryCrossentropy to compute loss
# make sure metrics=['accuracy] is used in the model.compile()

EPOCH = 10
model_history = model.fit(x=train_x, y=train_y, validation_data=(test_x, test_y), epochs=EPOCH)

In [ ]:
# ============================= step 4: plot loss/accuracy history ============================
loss = model_history.history['loss']
val_loss = model_history.history['val_loss']
acc = model_history.history['accuracy']  # make sure metrics=['accuracy] is used in the model.compile()
val_acc = model_history.history['val_accuracy']  # make sure metrics=['accuracy] is used in the model.compile()

epochs = range(EPOCH)

plt.figure()
plt.subplot(1, 2, 1)
plt.plot(epochs, loss, 'r-', label='Training loss')
plt.plot(epochs, val_loss, 'b-', label='Validation loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training/Validation Loss')
plt.ylim([0, 1])  # adjust y axis if your training/validation loss is above 1
plt.legend(loc='upper right')
plt.subplot(1, 2, 2)
plt.plot(epochs, acc, 'r--', label='Training accuracy')
plt.plot(epochs, val_acc, 'b--', label='Validation accuracy')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training/Validation Accuracy')
plt.ylim([0, 1])
plt.legend(loc='upper right')
plt.show()



In [ ]:

# ============================= step 5: prediction =============================
test_y_predicted = model.predict(test_x)

for i in range(3):
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
    display_grayscale_array(test_x[i], ax=ax1, title='Input image')
    display_segmented_image(test_y[i], ax=ax2, title='Ground truth')
    display_segmented_image(test_y_predicted[i], ax=ax3, title='Prediction', threshold=0.5)
    plt.show()
    # plot_class_masks(test_y[i], test_y_predicted[i], title='y target and y predicted sliced along the channel axis')
